<a href="https://colab.research.google.com/github/PradeepKumarM30/MLWithLargeData/blob/main/SGD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [85]:
import sklearn.datasets
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix,accuracy_score,f1_score,precision_score,recall_score,classification_report
import seaborn as sns
import numpy as np

In [86]:
df = pd.read_csv("/content/sample_data/loan.csv")
df.head()

,customer_id,disbursed_amount,interest,market,employment,time_employed,householder,income,date_issued,target,loan_purpose,number_open_accounts,date_last_payment,number_credit_lines_12
0,0,23201.5,15.4840,C,Teacher,<=5 years,RENT,84600.0,2013-06-11,0,Debt consolidation,4.0,2016-01-14,NaN
1,1,7425.0,11.2032,B,Accountant,<=5 years,OWNER,102000.0,2014-05-08,0,Car purchase,13.0,2016-01-25,NaN
2,2,11150.0,8.5100,A,Statistician,<=5 years,RENT,69840.0,2013-10-26,0,Debt consolidation,8.0,2014-09-26,NaN
3,3,7600.0,5.8656,A,Other,<=5 years,RENT,100386.0,2015-08-20,0,Debt consolidation,20.0,2016-01-26,NaN
4,4,31960.0,18.7392,E,Bus driver,>5 years,RENT,95040.0,2014-07-22,0,Debt consolidation,14.0,2016-01-11,NaN


In [87]:
df.tail(10)

,customer_id,disbursed_amount,interest,market,employment,time_employed,householder,income,date_issued,target,loan_purpose,number_open_accounts,date_last_payment,number_credit_lines_12
9990,9990,19200.0,14.6734,C,Statistician,>5 years,MORTGAGE,46865.0,2013-09-02,0,Debt consolidation,16.0,2015-12-16,NaN
9991,9991,20580.0,12.0442,C,Software developer,>5 years,RENT,47000.0,2015-08-14,0,Debt consolidation,6.0,2016-01-18,NaN
9992,9992,1470.0,12.5900,C,Statistician,<=5 years,RENT,23660.0,2015-12-19,0,Debt consolidation,6.0,2016-01-24,1.0
9993,9993,11640.0,13.3083,C,Other,<=5 years,MORTGAGE,57960.0,2015-01-17,0,Debt consolidation,6.0,2015-12-27,NaN
9994,9994,5520.0,10.6603,B,Accountant,<=5 years,RENT,30380.0,2015-11-17,0,Debt consolidation,12.0,2016-01-20,NaN
9995,9995,23750.0,11.0019,B,Civil Servant,<=5 years,MORTGAGE,98000.0,2010-01-14,0,Other,12.0,2010-11-03,NaN
9996,9996,11880.0,10.4923,B,Civil Servant,<=5 years,RENT,36270.0,2015-03-20,0,Debt consolidation,17.0,2016-01-26,NaN
9997,9997,19950.0,8.4364,B,Accountant,>5 years,OWNER,52250.0,2015-04-03,0,Debt consolidation,9.0,2015-12-12,NaN
9998,9998,4850.0,13.6409,C,Bus driver,>5 years,RENT,80040.0,2014-11-23,0,Debt consolidation,26.0,2016-01-01,NaN
9999,9999,25584.0,17.1456,D,Accountant,>5 years,MORTGAGE,72000.0,2015-01-19,0,Debt consolidation,5.0,2016-01-16,NaN


In [88]:
df.dtypes

customer_id                 int64
disbursed_amount          float64
interest                  float64
market                     object
employment                 object
time_employed              object
householder                object
income                    float64
date_issued                object
target                      int64
loan_purpose               object
number_open_accounts      float64
date_last_payment          object
number_credit_lines_12    float64
dtype: object

In [89]:
df["number_credit_lines_12"].isna().sum()

9762

In [90]:
df["disbursed_amount"].fillna(df["disbursed_amount"].mean())

0       23201.5
1        7425.0
2       11150.0
3        7600.0
4       31960.0
         ...   
9995    23750.0
9996    11880.0
9997    19950.0
9998     4850.0
9999    25584.0
Name: disbursed_amount, Length: 10000, dtype: float64

In [91]:
values = {'employment': 'Other'}
df.fillna(value=values,inplace=True)

In [92]:
values = {'time_employed': 'NA'}
df.fillna(value=values,inplace=True)

In [93]:

df["number_credit_lines_12"].fillna(df["number_credit_lines_12"].mean(),inplace=True)

In [94]:
df.head()

,customer_id,disbursed_amount,interest,market,employment,time_employed,householder,income,date_issued,target,loan_purpose,number_open_accounts,date_last_payment,number_credit_lines_12
0,0,23201.5,15.4840,C,Teacher,<=5 years,RENT,84600.0,2013-06-11,0,Debt consolidation,4.0,2016-01-14,1.785714
1,1,7425.0,11.2032,B,Accountant,<=5 years,OWNER,102000.0,2014-05-08,0,Car purchase,13.0,2016-01-25,1.785714
2,2,11150.0,8.5100,A,Statistician,<=5 years,RENT,69840.0,2013-10-26,0,Debt consolidation,8.0,2014-09-26,1.785714
3,3,7600.0,5.8656,A,Other,<=5 years,RENT,100386.0,2015-08-20,0,Debt consolidation,20.0,2016-01-26,1.785714
4,4,31960.0,18.7392,E,Bus driver,>5 years,RENT,95040.0,2014-07-22,0,Debt consolidation,14.0,2016-01-11,1.785714


In [95]:
df["market"].value_counts()

B    2846
C    2730
A    1673
D    1577
E    1174
Name: market, dtype: int64

In [96]:
df["market"] = df["market"].astype('category')
df["market_enc"] = df["market"].cat.codes


In [97]:
df["employment"] = df["employment"].astype('category')
df["employment_enc"] = df["employment"].cat.codes
df["time_employed"] = df["time_employed"].astype('category')
df["time_employed_enc"] = df["time_employed"].cat.codes
df['householder'] = df['householder'].apply(lambda val:1 if val == 'MORTGAGE' else (2 if val == 'RENT' else 3)) #point no 3
df["date_issued"] = df["date_issued"].astype('category')
df["date_issued_enc"] = df["date_issued"].cat.codes
df["loan_purpose"] = df["loan_purpose"].astype('category')
df["loan_purpose_enc"] = df["loan_purpose"].cat.codes
df["date_last_payment"] = df["date_last_payment"].astype('category')
df["date_last_payment_enc"] = df["date_last_payment"].cat.codes

In [98]:
df.head()

,customer_id,disbursed_amount,interest,market,employment,time_employed,householder,income,date_issued,target,loan_purpose,number_open_accounts,date_last_payment,number_credit_lines_12,market_enc,employment_enc,time_employed_enc,date_issued_enc,loan_purpose_enc,date_last_payment_enc
0,0,23201.5,15.4840,C,Teacher,<=5 years,2,84600.0,2013-06-11,0,Debt consolidation,4.0,2016-01-14,1.785714,2,10,0,821,1,1057
1,1,7425.0,11.2032,B,Accountant,<=5 years,3,102000.0,2014-05-08,0,Car purchase,13.0,2016-01-25,1.785714,1,0,0,1111,0,1068
2,2,11150.0,8.5100,A,Statistician,<=5 years,2,69840.0,2013-10-26,0,Debt consolidation,8.0,2014-09-26,1.785714,0,8,0,941,1,645
3,3,7600.0,5.8656,A,Other,<=5 years,2,100386.0,2015-08-20,0,Debt consolidation,20.0,2016-01-26,1.785714,0,5,0,1526,1,1069
4,4,31960.0,18.7392,E,Bus driver,>5 years,2,95040.0,2014-07-22,0,Debt consolidation,14.0,2016-01-11,1.785714,4,1,1,1179,1,1054


In [99]:
df.drop(columns=['employment', 'market','time_employed','date_issued','loan_purpose','date_last_payment'],inplace=True)
df.head()

,customer_id,disbursed_amount,interest,householder,income,target,number_open_accounts,number_credit_lines_12,market_enc,employment_enc,time_employed_enc,date_issued_enc,loan_purpose_enc,date_last_payment_enc
0,0,23201.5,15.4840,2,84600.0,0,4.0,1.785714,2,10,0,821,1,1057
1,1,7425.0,11.2032,3,102000.0,0,13.0,1.785714,1,0,0,1111,0,1068
2,2,11150.0,8.5100,2,69840.0,0,8.0,1.785714,0,8,0,941,1,645
3,3,7600.0,5.8656,2,100386.0,0,20.0,1.785714,0,5,0,1526,1,1069
4,4,31960.0,18.7392,2,95040.0,0,14.0,1.785714,4,1,1,1179,1,1054


In [100]:
df.drop(columns=['customer_id'],inplace=True)

In [101]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df[["disbursed_amount","interest","householder","income","number_open_accounts","number_credit_lines_12","market_enc","employment_enc","time_employed_enc","date_issued_enc","loan_purpose_enc","date_last_payment_enc"]] = scaler.fit_transform(df[["disbursed_amount","interest","householder","income","number_open_accounts","number_credit_lines_12","market_enc","employment_enc","time_employed_enc","date_issued_enc","loan_purpose_enc","date_last_payment_enc"]])

In [102]:
X_data = df.drop('target',axis=1)
Y_data = df['target']

In [103]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X_data,Y_data,test_size = 0.3 )

In [104]:
print("X TRaIN SIZE : ",X_train.shape," X TEST SIZE : ",X_test.shape,"Y TRaIN SIZE : ",Y_train.shape,"Y TRaIN SIZE : ",Y_test.shape)

X TRaIN SIZE :  (7000, 12)  X TEST SIZE :  (3000, 12) Y TRaIN SIZE :  (7000,) Y TRaIN SIZE :  (3000,)


In [105]:
#initialise weight and bias matrix
def initialize_with_zeros(dim):
   
    w = np.zeros((dim,1))
    b = 0
   
    assert(w.shape == (dim, 1))
    assert(isinstance(b, float) or isinstance(b, int))
    
    return w, b

In [106]:
#Sigmoid activation utility
def sigmoid(z):
   
    s = 1/(1+(np.exp(-z)))
    
    return s

In [107]:
def binary_activation(z):
  s = np.zeros(z.shape)
  rows = z.shape[0]
  cols = z.shape[1]
  for x in range(0, rows):
    for y in range(0, cols):
         if z[x,y]  >= 0.2: 
           s[x,y] = 1
         else :
           s[x,y] = 0
  return s

In [108]:
#forward and backward propogation
def propagate(w, b, X, Y):
    
    m = 1
    X = X.reshape(-1,1)
    A = sigmoid(np.dot(w.T,X)+b)          
    A = A.reshape(-1,1)
    cost = (-1/m) * (np.dot(Y,(np.log(A)).T) + np.dot(1-Y,(np.log(1-A)).T))                               
    dw = np.dot(X,(A-Y).T)/m
    db = (1/m) * np.sum(A-Y)

    assert(dw.shape == w.shape)
    assert(db.dtype == float)
    cost = np.squeeze(cost)
    assert(cost.shape == ())
    
    grads = {"dw": dw,
             "db": db}
    
    return grads, cost

In [109]:
#doing the computations
def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost = False):
    
    costs = []
    Y_val = Y.values
    Y_val = Y_val.reshape(-1,1)


    print("x    :",X.values.shape)
    for i in range(num_iterations):
      for row in range(0,7000):
        grads,cost = propagate(w,b,X.values[row,:],Y_val[row,:])
        dw = grads["dw"]
        db = grads["db"]
        
        w = w - (learning_rate * dw)
        b = b - (learning_rate * db)
        
        
        costs.append(cost)
    
    
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    
    return params, grads, costs

In [110]:
#Perceptron driver code
def perceptron(X_train, Y_train, num_iterations, learning_rate = 0.5):
   
    w,b = initialize_with_zeros(X_train.shape[1])

    parameters, grads, costs = optimize(w, b, X_train, Y_train, num_iterations, learning_rate)
    
    w = parameters["w"]
    b = parameters["b"]

    print(w.shape)
    
   # Y_prediction_test = predict(w, b, X_test)
    Y_prediction_train = predict(w, b, X_train)

    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train.values[0:7000])) * 100))
   # print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))

    
    d = {"costs": costs, 
         "Y_prediction_train" : Y_prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
    
    return d

In [111]:
#function to predict models performance
def predict(w, b, X):
    
    m = X.shape[0]
    Y_prediction = np.zeros((1,m))
   # w = w.reshape(X.shape[0], 1)
    
    A = binary_activation(np.dot(X.values[0:m,:],w)+b)
    Y_prediction = Y_prediction.T
    print("Y_prediction ",Y_prediction.shape)
    print("A ",A.shape)

    

    for i in range(m):
        Y_prediction[i,0] = 1 if A[i,0] > 0.52 else 0
    
    
    return Y_prediction

In [112]:
#the main driver code
print(" M PRADEEP KUMAR - 20MAI0030")
output_dict = perceptron(X_train,Y_train,2,1)
print(output_dict)

 M PRADEEP KUMAR - 20MAI0030
x    : (7000, 12)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in log
  


(12, 1)
Y_prediction  (7000, 1)
A  (7000, 1)
train accuracy: 98.71696326530612 %
{'costs': [array(0.69314718), array(1.09746865), array(0.40141883), array(0.2230395), array(2.81143897), array(5.73376878), array(0.15506123), array(0.00047162), array(0.00339262), array(0.05497014), array(0.01708816), array(0.00386777), array(0.15494868), array(0.0032337), array(0.1690564), array(0.1188229), array(0.00548707), array(0.03017608), array(0.00511892), array(0.04786625), array(0.01123407), array(0.06695876), array(0.00134551), array(0.02293291), array(0.10439705), array(4.9608049e-06), array(0.00468785), array(0.003921), array(0.0017544), array(0.02086972), array(0.00426929), array(0.03625128), array(0.00677255), array(0.01005419), array(0.00088501), array(0.00636318), array(0.03120486), array(0.01808022), array(0.01520901), array(0.00714203), array(0.06152998), array(0.02274037), array(0.00113107), array(0.03380879), array(0.00587963), array(0.00198854), array(5.98079705), array(0.52758093), 

In [71]:
#initialise weight and bias matrix
def initialize_with_zeros(dim):
   
    w = np.zeros((dim,1))
    b = 0
   
    assert(w.shape == (dim, 1))
    assert(isinstance(b, float) or isinstance(b, int))
    
    return w, b

In [72]:
#Sigmoid activation utility
def sigmoid(z):
   
    s = 1/(1+(np.exp(-z)))
    
    return s

In [73]:
def propagate(w, b, X, Y):
    
    m = X.shape[1]
    
    A = sigmoid(np.dot(w.T,X)+b)                                   
    cost = (-1/m) * (np.dot(Y,(np.log(A)).T) + np.dot(1-Y,(np.log(1-A)).T))                               
    dw = np.dot(X,(A-Y).T)/m
    db = (1/m) * np.sum(A-Y)

    assert(dw.shape == w.shape)
    assert(db.dtype == float)
    cost = np.squeeze(cost)
    assert(cost.shape == ())
    
    grads = {"dw": dw,
             "db": db}
    
    return grads, cost

In [80]:
def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost = False):
    
    costs = []
    
    for i in range(num_iterations):
        grads, cost = propagate(w,b,X.values,Y)
        
        dw = grads["dw"]
        db = grads["db"]
        
        w = w - (learning_rate * dw)
        b = b - (learning_rate * db)
        
        if i % 100 == 0:
            costs.append(cost)
        
        if print_cost and i % 10 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
    
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    
    return params, grads, costs

In [81]:
def perceptron(X_train, Y_train, num_iterations, learning_rate = 0.5):
   
    w,b = initialize_with_zeros(X_train.shape[0])

    parameters, grads, costs = optimize(w, b, X_train, Y_train, num_iterations, learning_rate)
    
    w = parameters["w"]
    b = parameters["b"]
    
#    Y_prediction_test = predict(w, b, X_test)
    Y_prediction_train = predict(w, b, X_train)

    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
#    print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))

    
    d = {"costs": costs, 
         "Y_prediction_train" : Y_prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
    
    return d

In [82]:
def predict(w, b, X):
    
    m = X.shape[1]
    Y_prediction = np.zeros((1,m))
    w = w.reshape(X.shape[0], 1)
    
    A = sigmoid(np.dot(w.T,X)+b)
    
    for i in range(A.shape[1]):
        Y_prediction[:,i] = 1 if A[:,i] > 0.52 else 0
    
    assert(Y_prediction.shape == (1, m))
    
    return Y_prediction

In [83]:
print(" M PRADEEP KUMAR - 20MAI0030")
output_dict = perceptron(X_train,Y_train,2000,0.2)
print(output_dict)

 M PRADEEP KUMAR - 20MAI0030


ValueError: ignored